In [2]:
import os
import torch
from torchvision import transforms
from PIL import Image
import cv2
import numpy as np

In [47]:
import os
from PIL import Image
from torchvision import transforms

folder_path = "/Users/jmac/Desktop/clg/BI-RADS/"
image_files = os.listdir(folder_path)

image_tensors = []
image_files = sorted(
    [f for f in image_files if f.endswith(('.jpg', '.jpeg', '.png'))]
)

for filename in image_files:
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(folder_path, filename)
        print(image_path)       
        image = Image.open(image_path)        
        transform = transforms.ToTensor()
        image_tensor = transform(image)
        image_tensors.append(image_tensor)

image_array = np.stack(image_tensors)

print(image_array.shape)        


/Users/jmac/Desktop/clg/BI-RADS/BI-RADS 1.png
/Users/jmac/Desktop/clg/BI-RADS/BI-RADS 2.png
/Users/jmac/Desktop/clg/BI-RADS/BI-RADS 3.png
/Users/jmac/Desktop/clg/BI-RADS/BI-RADS 4.png
/Users/jmac/Desktop/clg/BI-RADS/BI-RADS 5.png
(5, 3, 224, 224)


In [55]:
import torch
import torchvision.models as models
model_path = '/Users/jmac/Desktop/clg/resnet50BasedModel/best.pt'
#model_path='/Users/jmac/Desktop/clg/resnet50BasedModel/last.pt'
# Instantiate the model (e.g., ResNet50 based model)
model = models.resnet50()  # Adjust this line according to your model architecture
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 512)

# Load the checkpoint
checkpoint = torch.load(model_path, map_location=torch.device('cpu'))

# Extract the model state dictionary from the checkpoint
state_dict = checkpoint['model_state_dict']
new_state_dict = {k.replace('model.', ''): v for k, v in state_dict.items()}

# Load the state dictionary into the model
model.load_state_dict(new_state_dict)
model.eval()

# List to store the output
output = []

# Assume image_tensors is a list of tensors representing images
for input_tensor in image_tensors:
    input_batch = input_tensor.unsqueeze(0)  # Add batch dimension

    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    with torch.no_grad():
        out = model(input_batch)
        output.append(out)

# Tensor of shape 1000, with confidence scores over ImageNet's 1000 classes
print("Output shape before softmax:", output[0].shape)


Output shape before softmax: torch.Size([1, 512])


In [56]:
import torch.nn.functional as F
print( F.cosine_similarity(output[2], output[0], dim=1))
print( F.cosine_similarity(output[2], output[1], dim=1))
print( F.cosine_similarity(output[2], output[3], dim=1))
print( F.cosine_similarity(output[2], output[4], dim=1))
#0 near 1 , far 2,4,3
#1 near 0 , far 2,3,4
#2 near 1 , far 4,3,0
#3 near 4 , far 3,0,1
#4 near 3 , far 2,0,1( cos xếp tăng dần)
#2: 3-1-0-4

tensor([0.0762])
tensor([0.0917])
tensor([0.1896])
tensor([-0.0303])


In [57]:
import torch.nn.functional as F
print( F.cosine_similarity(output[1], output[0], dim=1))
print( F.cosine_similarity(output[1], output[2], dim=1))
print( F.cosine_similarity(output[1], output[3], dim=1))
print( F.cosine_similarity(output[1], output[4], dim=1))
# 1: 0-3-2-4

tensor([0.1800])
tensor([0.0917])
tensor([0.1746])
tensor([0.0262])


In [58]:
import torch.nn.functional as F
print( F.cosine_similarity(output[3], output[0], dim=1))
print( F.cosine_similarity(output[3], output[1], dim=1))
print( F.cosine_similarity(output[3], output[2], dim=1))
print( F.cosine_similarity(output[3], output[4], dim=1))
# 3: 2-1-0- 4

tensor([0.1213])
tensor([0.1746])
tensor([0.1896])
tensor([0.0360])


In [59]:
import torch.nn.functional as F
print( F.cosine_similarity(output[4], output[0], dim=1))
print( F.cosine_similarity(output[4], output[1], dim=1))
print( F.cosine_similarity(output[4], output[2], dim=1))
print( F.cosine_similarity(output[4], output[3], dim=1))
#4: 3-1-0-2

tensor([0.0217])
tensor([0.0262])
tensor([-0.0303])
tensor([0.0360])


In [60]:
import torch.nn.functional as F
print( F.cosine_similarity(output[0], output[1], dim=1))
print( F.cosine_similarity(output[0], output[2], dim=1))
print( F.cosine_similarity(output[0], output[3], dim=1))
print( F.cosine_similarity(output[0], output[4], dim=1))
# 0: 1-3-2-4 (sắp xếp theo cos giảm dần)
# 1: 0-3-2-4
# 2: 3-1-0-4
# 3: 2-1-0-4
# 4: 3-1-0-2
# 0->4 là mức độ từ 1->5



tensor([0.1800])
tensor([0.0762])
tensor([0.1213])
tensor([0.0217])
